# 02-rag.ipynb

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# %pip install pypdf langchain-community

## 저장 파트

In [ ]:
# 1. Document Load (PDF)
# 지원하는 문서 로더: https://docs.langchain.com/oss/python/integrations/document_loaders

from langchain_community.document_loaders import PyPDFLoader

# 불러올 파일 위치
file_path = './nke-10k-2023.pdf'

# 대상 pdf 를 변환해줄 로더
loader = PyPDFLoader(file_path)

# 로더가 pdf를 python에서 쓸 수 있도록 변환(pdf 1page -> 1 Document)
docs = loader.load()

print(len(docs))  # 원본 pdf 페이지수가 나옴

In [ ]:
# 2. Splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Document 를 잘라줄 스플리터
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

# 쪼개기
chunks = text_splitter.split_documents(docs)

print(len(chunks))  # 전체 chunk 개수
print(chunks[0].page_content)  # 첫번째 청크의 원본 텍스트 내용

In [ ]:
# 3. Embedding (숫자로 바꾸기)
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# 아래는 테스트용 (실제 텍스트 -> 벡터로 바뀌는 과정)
v1 = embeddings.embed_query(chunks[0].page_content)  # 청크1 벡터로 변환
v2 = embeddings.embed_query(chunks[1].page_content)  # 청크2 벡터로 변환

# 차원수는 같아야 한다.
print(len(v1) == len(v2))
print(v1[:10])  # 벡터 눈으로 확인하기

In [ ]:
# 4. Vector Store 에 저장하기
from langchain_core.vectorstores import InMemoryVectorStore

# 테스트/개발용 메모리 벡터스토어
vector_store = InMemoryVectorStore(embeddings)

# pdf 쪼개놓은 chunks 를 벡터스토어에 저장 (저장 후 id들이 나옴)
ids = vector_store.add_documents(documents=chunks)

## 검색 파트

In [ ]:
# 벡터스토어 -> 검색기로 활용
retriever = vector_store.as_retriever(
    search_type='similarity',  # 검색방식: 유사도
    search_kwargs={'k': 3}     # 결과개수: 3개
)

# 검색
retriever.invoke('나이키의 미국 영업점 개수?')

## PDF RAG를 Agent 에 통합

In [ ]:
# 검색기(retriever)를 Tool(함수)로 만들기

# 검색어(query)를 인자로 받음
def search_vectorstore(query: str) -> str:
    """Retrieve info to help answer a query about Nike"""
    # 검색기 대신 벡터스토어 바로 활용하기 (chunk 2개만 검색)
    docs = vector_store.similarity_search(query, k=2)
    result = ''

    for doc in docs:
        result += doc.page_content + '\n\n'

    return result


print(search_vectorstore('나이키 영업점 개수'))

In [ ]:
from langchain.agents import create_agent

prompt = """너는 2023 나이키 10k 보고서를 검색하는 도구를 다룰 수 있어. 
사용자 질문에 답변하기 위해 필요하면 사용해. 경제분석 전문가처럼 답변해."""


agent = create_agent(
    model="openai:gpt-4.1-mini",
    tools=[search_vectorstore],
    system_prompt=prompt
)

In [ ]:
content = "나이키 영업점 숫자와 각 영업점 평균 매출액이 궁금함."

agent.invoke(
    {
        "messages": [
            {"role": "user", "content": content}
        ]
    }
)

## Web문서(HTML) RAG + Agent

In [ ]:
# % pip install beautifulsoup4

In [2]:
# HTML은 문서 본문 외에 필요하지 않은 내용이 많다. 전처리가 필요하다!
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 전처리기
bs4_strainer = bs4.SoupStrainer(class_=('post-title', 'post-header', 'post-content'))
# 로더
loader = WebBaseLoader(
    web_path="https://lilianweng.github.io/posts/2023-06-23-agent/",
    bs_kwargs={'parse_only': bs4_strainer},  # 처리기 넣기
)

docs = loader.load()
# 문서 페이지 수, 총 글자수
print( len(docs), len(docs[0].page_content) )


USER_AGENT environment variable not set, consider setting it to identify your requests.


1 43047


### Todos
1. Web Base Loader 로 불러온 docs Split
2. Split 된 내용들을 OpenAI Embedding Model 을 활용하여 Vectorstore에 저장
3. Agent 에 통합하기

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Split
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
chunks = text_splitter.split_documents(docs)
print(len(chunks))

# Embedding
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# Store
vector_store = InMemoryVectorStore(embeddings)
ids = vector_store.add_documents(documents=chunks)

63


In [11]:
from langchain.agents import create_agent

def search_vectorstore(query: str) -> str:
    """Retrieve info to help answer a user's query """
    docs = vector_store.similarity_search(query, k=4)
    result = ''

    for doc in docs:
        result += doc.page_content + '\n\n'

    return result


prompt = """사용자 질문에 답변하기 위한 검색하는 도구를 다룰 수 있어. 
사용자 질문에 답변하기 위해 사용해. 검색한 내용을 기반으로 답변해"""


# Agent 에 통합하기
agent = create_agent(
    model="openai:gpt-4.1-mini",
    tools=[search_vectorstore],
    system_prompt=prompt
)

In [12]:
content = "자율 Agent라는게 결국 무엇을 의미하는가"

agent.invoke(
    {
        "messages": [
            {"role": "user", "content": content}
        ]
    }
)

{'messages': [HumanMessage(content='자율 Agent라는게 결국 무엇을 의미하는가', additional_kwargs={}, response_metadata={}, id='62891f02-fda0-40c6-a47d-57ee4aff020d'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 98, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a391f2cee0', 'id': 'chatcmpl-DCh14AgEd8qy6fRRTfEIfSuUgdSLa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c8e85-d94c-7da3-8a08-58ebfca9c6d9-0', tool_calls=[{'name': 'search_vectorstore', 'args': {'query': '자율 Agent 의미'}, 'id': 'call_2jG4PDGkdNKA1ZuV61es0qwx', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 